In [46]:
import requests

In [47]:
port_response = requests.get("https://api.tidesandcurrents.noaa.gov/mdapi/prod/webapi/ports.json").text

In [48]:
import json

In [49]:
port_response_dict = json.loads(port_response)

In [50]:
from collections import defaultdict

# Useful arrays 

In [51]:
products = {
    "waterlevel" : "water_level",
    "wind" : "wind",
    "currents" : "currents",
    "airTemp" : "air_temperature",
    "waterTemp" : "water_temperature",
    "baroPressure" : "air_pressure",
    "salinity" : "salinity",
    "airGap" : "air_gap",
    "relHumidity" : "humidity",
    "visibility" : "visibility",
}

In [52]:
feed_template = {
    "Dataset Name" : "Water Levels / $STATION$, $PORT$",
    # Water Levels / Kiptopeke Beach, Chesapeake Bay South
    "Latitude" : "$LAT$",
    # 37.1652
    "Longitude" : "$LON$",
    # -75.9884
    "Notes" : "$STATION$, Sensor $ID$",
    # Kiptopeke Beach, Sensor 8632200
    "Dataset URL" : "$URL$"
    # https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?date=latest&station=8632200&product=water_level&datum=STND&time_zone=gmt&units=english&format=json&application=TERBINE_IO
}

In [53]:
def replace(item, feed_dict):
    if type(item) is str:
        for replace_key, replace_val in feed_dict.items():
            item = item.replace(replace_key, str(replace_val))
    return item

In [54]:
def dict_replace(feed_template, feed_dict):
    new_feed = feed_template.copy()
    for key in new_feed.keys():
        new_feed[key] = replace(new_feed[key], feed_dict)
    return new_feed

In [55]:
def getStationsURL(port):
    return (
        "https://api.tidesandcurrents.noaa.gov/mdapi/prod/webapi/portsstation.json?ports=" 
        + port
        )

In [56]:
def getStations(port):
    station_response = requests.get(getStationsURL(port)).text
    return json.loads(station_response)

In [57]:
def getProductURL(product, id):
    return (
        "https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?date=latest&station="
        + id
        + "&product="
        + product
        + "&datum=STND&time_zone=gmt&units=english&format=json&application=TERBINE_IO"
        )

In [58]:
def getProduct(product, id):
    station_response = requests.get(getProductURL(product, id)).text
    return json.loads(station_response)

In [59]:
def requestProductsFromStation(station, requested_products, print_err=False):
    available_products = []
    #print(station['stationID'])

    for key in station.keys():
        if key in products.keys():

            product_bool = station[key]
            product = products[key]
        
            if product_bool and product in requested_products:
                product_response = getProduct(product, station['stationID'])

                if 'error' in product_response.keys():
                    if print_err:
                        print("There was an error with product '", key, "(", product, ")' at station", station['stationID'], "(", station['label'],")")
                        print('\t', "ERROR: ",product_response['error']['message'])
                else:
                    #print(product_response['data'][0])
                    available_products.append(product)

    return available_products

## The products we want to create a Feed for

In [60]:
requested_products = ["water_level"]

In [61]:
def feed_write(path, feed_dict):
    with open(path, 'a') as f:
        # clear file
        for val in feed_dict.values():
            f.write(val)
            f.write('\n')
        f.write("\n\n")
    f.close()

In [62]:
def clear_file(path):
    with open(path, 'w') as f:
        f.truncate(0)
    f.close()

In [63]:
def idValid(id, id_json):
    for a in id_json.values():
        if id in a:
            return False
    return True

In [64]:
with open('json_data.json') as json_file:
    data = json.load(json_file)
    id_json = defaultdict(list, data)

In [65]:
path = "readme.md"
clear_file(path)

used_stations = []
used_ports = []

for port in port_response_dict['portsList'][11:14]:
    #print(port['PORTSCode'], ":")
    station_response_dict = getStations(port['PORTSCode'])

    for station in station_response_dict['portsStationList']:
        sID = station['stationID']

        if sID.isnumeric() and idValid(sID, id_json):
            available_products = requestProductsFromStation(station, requested_products)

            for product in available_products:
            # use each product to create a feed
                if sID not in used_stations:
                    used_stations.append(sID)
                if port['PORTSCode'] not in used_ports:
                    used_ports.append(port['PORTSCode'])
                
                feed_dict = {
                    "$PORT$": port['PORTSName'],
                    "$STATION$": station['label'],
                    "$LAT$": station['lat'],
                    "$LON$": station['lng'],
                    "$ID$": sID,
                    "$URL$": getProductURL(product, sID),
                    # some names have a /, replace with a space
                }
                var_replace = {
                    " PORTS" : "",
                    "/" : " ",
                }

                id_json[port['PORTSCode']].append(sID)
                # track used IDS

                feed_dict['$PORT$'] = replace(feed_dict['$PORT$'], var_replace)
                # remove some bad chars from the values of these vars
                feed_formatted = dict_replace(feed_template, feed_dict)
                # format feed using above dict
                feed_write(path, feed_formatted)
                # write feed for output


In [66]:
    f = open(path, "a")
    f.write(str(len(used_ports)) + " Port(s), " + str(len(used_stations)) + " Station(s).")
    # print line for #Searcher-Submissions
    f.write('\n')
    f.close()

In [67]:
with open('json_data.json', 'w') as outfile:
    json.dump(id_json, outfile, indent=4)